In [35]:
def normalize(df, min, maxX, maxY):
  df['box2d.x1'] = (df['box2d.x1'] - min) / (maxX - min)
  df['box2d.x2'] = (df['box2d.x2'] - min) / (maxX - min)
  df['box2d.y1'] = (df['box2d.y1'] - min) / (maxY - min)
  df['box2d.y2'] = (df['box2d.y2'] - min) / (maxY - min)
  return df

In [36]:
import pandas as pd
import numpy as np

df = pd.read_csv('mot_labels.csv')
df['category'] = df['category'].map({'car': 1, 'pedestrian': 0, 3: 3}).fillna(3).astype(int)

ids = df[["id"]].drop_duplicates()

print(ids)

list = []

# count the occurrences of each unique value in the 'fruit' column
counts = df['id'].value_counts()

# get the value that appears the most number of times
most_common_value = counts.idxmax()

print(most_common_value)


path = df.loc[df['id'] == most_common_value] [['category','box2d.x1','box2d.x2','box2d.y1','box2d.y2']]
list.append(path)

minLength = 100
min_value = 0
max_xValue = 1280
max_yValue = 720

i = 0
LENGTH = 0
for ind in ids.index:
  
  if i>5000:
    break
  
  value = ids['id'][ind]
  
  path = df.loc[df['id'] == value] 
  #print(len(path))
  if len(path)<minLength:
    continue


  #print("path of: "+str(value))
  #print(path)
  
  
  path = path[['category','box2d.x1','box2d.x2','box2d.y1','box2d.y2']]
  path = normalize(path, min_value, max_xValue, max_yValue)
  list.append(path)
  i+=1

print("Sample")
print(list[0])

#print(list[0])


C:\Users\Frank\AppData\Local\Temp\ipykernel_23456\1376049851.py:4: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('mot_labels.csv')


              id
0        89537.0
1        89538.0
2        89539.0
3        89540.0
4        89541.0
...          ...
2890739  70589.0
2890768  70590.0
2890792  70591.0
2890813  70592.0
2890817  70593.0

[112820 rows x 1 columns]
87944.0
Sample
         category    box2d.x1     box2d.x2    box2d.y1    box2d.y2
1474762         3  587.492687   779.066388  305.422741  462.857143
1474781         3  594.218386   778.482735  305.396697  460.578130
1474800         3  595.887547   786.900583  314.925274  465.725040
1474818         3  592.478134   796.093294  321.166181  483.848397
1474838         3  592.478134   806.588921  306.472303  475.451895
...           ...         ...          ...         ...         ...
1476430         3  638.925608  1278.570509    0.000000  720.000000
1476435         3  639.019531  1278.563702    0.000000  720.000000
1476440         3  639.113453  1278.556895    0.000000  720.000000
1476445         3  639.207376  1278.550088    0.000000  720.000000
1476450         3

In [37]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    
    #print(data_raw)
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    #print(data)
    data = np.array(data)
    test_set_size = int(np.round(0.2*data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    #print(data.shape)
    #print(test_set_size)
    #print(train_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [38]:
import torch
import torch.nn as nn

lookback = 10  # choose sequence length

In [39]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [40]:
input_dim = 5
hidden_dim = 32
num_layers = 2
output_dim = 5
num_epochs = 500

In [41]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [42]:
print(list[0])

         category    box2d.x1     box2d.x2    box2d.y1    box2d.y2
1474762         3  587.492687   779.066388  305.422741  462.857143
1474781         3  594.218386   778.482735  305.396697  460.578130
1474800         3  595.887547   786.900583  314.925274  465.725040
1474818         3  592.478134   796.093294  321.166181  483.848397
1474838         3  592.478134   806.588921  306.472303  475.451895
...           ...         ...          ...         ...         ...
1476430         3  638.925608  1278.570509    0.000000  720.000000
1476435         3  639.019531  1278.563702    0.000000  720.000000
1476440         3  639.113453  1278.556895    0.000000  720.000000
1476445         3  639.207376  1278.550088    0.000000  720.000000
1476450         3  639.301298  1278.543281    0.000000  720.000000

[263 rows x 5 columns]


In [43]:
import time


hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for x in range(len(list)):
    
    dataf = list[x]
    
    if dataf.empty or dataf.shape[0]<= lookback:
        continue
    
    x_train, y_train, x_test, y_test = split_data(list[x], lookback)
    #print('x_train.shape = ',x_train.shape)
    #print('y_train.shape = ',y_train.shape)
    #print('x_test.shape = ',x_test.shape)
    #print('y_test.shape = ',y_test.shape)

    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
    y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
    
    for t in range(num_epochs):
        y_train_pred = model(x_train)

        loss = criterion(y_train_pred, y_train_lstm)
        #print("Epoch ", t, "MSE: ", loss.item())
        hist[t] = loss.item()

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print("Data ", x, "MSE: ", loss.item())
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))


Data  0 MSE:  346053.09375
Data  1 MSE:  0.0011355120223015547
Data  2 MSE:  0.002279080916196108
Data  3 MSE:  0.006641029380261898
Data  4 MSE:  0.010869656689465046
Data  5 MSE:  0.007913590408861637
Data  6 MSE:  0.0011687068035826087
Data  7 MSE:  0.012292781844735146
Data  8 MSE:  0.008860981091856956
Data  9 MSE:  0.001156833372078836
Data  10 MSE:  0.0044028921984136105
Data  11 MSE:  0.0021672884467989206
Data  12 MSE:  0.00020596316608134657
Data  13 MSE:  0.00015720933151897043
Data  14 MSE:  0.00047017508768476546
Data  15 MSE:  0.0002882126718759537
Data  16 MSE:  0.0013006660155951977
Data  17 MSE:  0.00011070316395489499
Data  18 MSE:  0.0007452820427715778
Data  19 MSE:  0.0032278168946504593
Data  20 MSE:  0.001342674484476447
Data  21 MSE:  0.004392622504383326
Data  22 MSE:  0.00012462309678085148
Data  23 MSE:  0.0002715075097512454
Data  24 MSE:  0.0001511123264208436
Data  25 MSE:  0.00015117874136194587
Data  26 MSE:  4.113422619411722e-06
Data  27 MSE:  0.000217

In [ ]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:

print(x_test)

print(model(x_test))

tensor([[[  1.0000, 747.0208, 766.1432, 343.3718, 353.3487],
         [  1.0000, 746.1894, 761.1547, 323.4180, 335.0577],
         [  1.0000, 741.2009, 756.9977, 319.2610, 333.3949],
         [  1.0000, 734.5497, 752.8406, 314.2725, 329.2379],
         [  1.0000, 727.8984, 743.6951, 315.1039, 328.4065],
         [  1.0000, 719.5843, 737.8753, 321.7552, 337.5520],
         [  1.0000, 714.5958, 731.2240, 317.5981, 332.5635],
         [  1.0000, 703.7875, 722.9099, 314.2725, 329.2379],
         [  1.0000, 701.2933, 717.0901, 320.9238, 338.3834]],

        [[  1.0000, 746.1894, 761.1547, 323.4180, 335.0577],
         [  1.0000, 741.2009, 756.9977, 319.2610, 333.3949],
         [  1.0000, 734.5497, 752.8406, 314.2725, 329.2379],
         [  1.0000, 727.8984, 743.6951, 315.1039, 328.4065],
         [  1.0000, 719.5843, 737.8753, 321.7552, 337.5520],
         [  1.0000, 714.5958, 731.2240, 317.5981, 332.5635],
         [  1.0000, 703.7875, 722.9099, 314.2725, 329.2379],
         [  1.0000, 70

In [ ]:
def unNormalize(df, min, maxX, maxY):
    for i in range(df.shape[0]):
        
        if(df.ndim ==3):
            for j in range(df.shape[1]):
                df[i][j][1] = (df[i][j][1]  + min) * (maxX - min)
                df[i][j][2] = (df[i][j][2]  + min) * (maxX - min)
                df[i][j][3] = (df[i][j][3]  + min) * (maxY - min)
                df[i][j][4] = (df[i][j][4]  + min) * (maxY - min)
        
        if(df.ndim ==2):
            df[i][1] = (df[i][1]  + min) * (maxX - min)
            df[i][2] = (df[i][2]  + min) * (maxX - min)
            df[i][3] = (df[i][3]  + min) * (maxY - min)
            df[i][4] = (df[i][4]  + min) * (maxY - min)
    #df['box2d.x1'] = (df['box2d.x1'] - min) / (maxX - min)
    #df['box2d.x2'] = (df['box2d.x2'] - min) / (maxX - min)
    #df['box2d.y1'] = (df['box2d.y1'] - min) / (maxY - min)
    #df['box2d.y2'] = (df['box2d.y2'] - min) / (maxY - min)
    return df

In [ ]:
print("x_test")
print(x_test)
unNorm = unNormalize(x_test,  min_value, max_xValue, max_yValue)

print(unNorm)

prediction = model(x_test)
print("prediction:")
print(prediction.shape)

unNormPred = unNormalize(prediction,  min_value, max_xValue, max_yValue)

print(unNormPred)


#huh = unNormalize(x_test[0], min_value, max_xValue, max_yValue)
#print(huh)